In [62]:
# graph_comparison(graph, date, start_date, end_date, window, plotting, percentile, scope)

import matplotlib as plt
import datetime as dt
import numpy as np

def sortkey2(e):
    return e[1]

def daily_no_avg(graph, start, date, end_date, start2, scope):
    # returns list of that week average over all years
    date_data = []
    for date in list(graph.keys()):
        date_data.append([dt.datetime(int(date[0:4]),int(date[5:7]),int(date[8:10])), graph[date]])
    total_data = []
    # just take first day and next first day
    while start.year < int(end_date):
        lst2 = []
        accum = 0
        for i in range(len(date_data)):
            if date_data[i][0] >= start and date_data[i][0] <= start + dt.timedelta(days=scope):
                lst2.append(date_data[i])
                accum += date_data[i][1]
        if len(lst2) >= 2:
            total_data.append([start2.year, accum/len(lst2)])
        start = dt.datetime(start.year + 1, start.month, start.day)
        start2 = dt.datetime(start2.year + 1, start2.month, start2.day)
    return total_data

def graph_comparison(graph, date, start_date, end_date, window, plotting, percentile, scope):
    if len(date) == 5:
        start = dt.datetime(int(start_date), int(date[0:2]), int(date[3:5]))
        start2 = start + dt.timedelta(weeks=window)
        end2 = start2 + dt.timedelta(days=scope)
    first_set = daily_no_avg(graph[0], start, date, end_date, start, scope)
    second_set = daily_no_avg(graph[1], start2, date, end_date, start, scope)
    plottable = []
    for i in first_set:
        for j in second_set:
             if j[0] == i[0]:
                  if i[1] != 0:
                    plottable.append([i[0], j[1] - i[1]])
    dct = {}
    plottable.sort(key=sortkey2)
    for i in range(len(plottable)):
        if plotting:
            dct[(str(plottable[i][0]))[2:4]] = plottable[i][1]
        else:
            dct[(str(plottable[i][0]))] = plottable[i][1]
    if plotting:
        plt.bar(dct.keys(), dct.values(), color = "green", width = 0.8)
        plt.xlabel("Year")
        plt.ylabel("Value")
        plt.title("Streamflow (%s) versus period after" % (date))
        plt.show()
    else:
        if len(dct) > 0:
            if percentile != 0:
                a = list(dct.values())
                a = np.array(a)
                p = np.percentile(a, percentile)  # return 50th percentile, i.e. median.
                return p
            else:
                # [mean, median, interquartile range,standard deviation]
                b = list(dct.values())
                b = np.array(b)
                return [list(dct.values()), 0, 0, 0, 0,start.strftime("%m/%d"), end2.strftime("%m/%d"), list(dct.keys())]
                #return [list(dct.values()), np.mean(b), np.median(b), np.percentile(b, 75) - np.percentile(b, 25), np.std(b),start.strftime("%m/%d"), end2.strftime("%m/%d"), list(dct.keys())]
        else:
            return False

In [1]:
# using_all_weeks(date, end_date, percent, scope, window)
def sortkey(e):
    return e[0]

def using_all_weeks(date, end_date, percent, scope, window):
    start_date = 1980
    # CHANGE THIS TO FILENAME VVVVVVVVVVVVVV
    graph = json.load(open('{}soil_catskills.json'.format(outdir), 'r'))
    date = dt.datetime(start_date, int(date[0:2]), int(date[3:5]))
    total_data = []
    date_data = []
    for week in range(53):
        date_data.append({})
    for stat in graph:
        new_date = dt.datetime(int(stat[0:4]),int(stat[5:7]),int(stat[8:10]))
        date_data[((new_date.timetuple().tm_yday - 1) // 7)][stat] = graph[stat]
    for week in range(52):
        date2 = date + dt.timedelta(weeks=week)
        # investigate looking across year
        #Convert to string
        mo = str(date2.month)
        da = str(date2.day)
        if len(mo) == 1:
            mo = "0" + mo
        if len(da) == 1:
            da = '0' + da
        temp1 = {}
        temp2 = {}
        temp1 = {**temp1,**date_data[week]}
        if week + window < 52:
            temp2 = {**temp2,**date_data[week + window]}
        else:
            temp2 = {**temp2,**date_data[(52 - week + window)]}
        if scope // 7 > 1:
            for additional_week in range((scope // 7) + 1):
                if additional_week + week < 53:
                    temp1 = {**temp1, **date_data[week + additional_week]}
                else:
                    temp1 = {**temp1, **date_data[week + additional_week - 52]}
                if additional_week + week + (scope // 7) < 52:
                    temp2 = {**temp2, **date_data[week + additional_week + (scope // 7)]}
                else:
                    temp2 = {**temp2, **date_data[week + additional_week - 52 + (scope // 7)]}
        # COMBINE NECESSARY WEEKS FOR ANALYSIS
        info = (graph_comparison([temp1, temp2], f"{mo}-{da}", start_date, end_date, window, False, 0, scope))
        if info:
            for i in range(len(info[0])):
                total_data.append([info[0][i],f"{info[7][i]}/{(info[5])} - {info[7][i]}/{info[6]}"])
    total_data.sort(key=sortkey)
    templst = []
    for i in (total_data[0:10]):
        templst.append(f'{i[0]:.4f}, {i[1]}')
    print(templst)
    return total_data[0:(int(int(len(total_data) / (100 / percent))))]

In [64]:
# To convert retro_timeseries.json to a file readable by my code, un-comment and run this block

'''graph = json.load(open('{}retro_timeseries.json'.format('indir/'), 'r'))
lst = ["buffalo", "adirondacks", "catskills", "long_island"]
for region in lst:
    dct = {}
    for item in range(len(graph[region])):
        date = graph["dates"][item]
        date2 = f"{date[6:]}-{date[:2]}-{date[3:5]}"
        dct[date2] = graph[region][item]
    outfilename = '%ssoil_%s.json'%(outdir, region)
    outfile = open(outfilename, "w")
    json.dump(dct, outfile)
    outfile.close()'''

'graph = json.load(open(\'{}retro_timeseries.json\'.format(\'indir/\'), \'r\'))\nlst = ["buffalo", "adirondacks", "catskills", "long_island"]\nfor region in lst:\n    dct = {}\n    for item in range(len(graph[region])):\n        date = graph["dates"][item]\n        date2 = f"{date[6:]}-{date[:2]}-{date[3:5]}"\n        dct[date2] = graph[region][item]\n    outfilename = \'%ssoil_%s.json\'%(outdir, region)\n    outfile = open(outfilename, "w")\n    json.dump(dct, outfile)\n    outfile.close()'

In [65]:
# graphing(data, end_date, percentile)
MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', "Dec"]
import matplotlib.pyplot as plt

def graphing(data, end_date, percentile):
    year_accum = []
    year_accum2 = []
    for i in range(int(end_date)-1980):
        year_accum.append(0)
        year_accum2.append(0)
    month_accum = [0,0,0,0,0,0,0,0,0,0,0,0]
    for lst in data:
            month_accum[int(lst[1][5:7]) - 1] += 1

    plt.bar(MONTHS, month_accum, color ="blue")
    plt.title(f"Soil Moisture: {percentile} percent between 1979-{end_date}")
    plt.xlabel("Month")
    plt.ylabel("Number of Occurances")
    plt.show()
    if percentile == 10:
        print(f"1%: {data[int(len(data[0]) / 100)][0]:.4f}")
        print(f"2%: {data[int(len(data[0]) / 50)][0]:.4f}")
        print(f"5%: {data[int(len(data) / 20)][0]:.4f}")
        print(f"10%: {data[int(len(data) / 10)][0]:.4f}")

        for lst in data[0:int(len(data) / (10))]:
            year_accum[int(lst[1][0:4]) - int(1980)] += 1

        plt.bar((range(int(1980), int(end_date))), year_accum, color ="green")
        plt.title(f"Occurances across years: 1 percent between {1980}-{end_date}")
        plt.xlabel("Year")
        plt.ylabel("Number of Occurances")
        plt.show()

In [66]:
# Driver (same as evaporation)

outdir = "outdir/"
ryear, rmonth, rday = 2023, 2, 2
percentile = 1
scope = 7
window = 4
print("4 weeks")
all_data = using_all_weeks("01-01", ryear, percentile, scope, 4)
graphing(all_data, ryear, percentile)

print("3 weeks")
all_data = using_all_weeks("01-01", ryear, percentile, scope, 3)
graphing(all_data, ryear, percentile)

print("2 weeks")
all_data = using_all_weeks("01-01", ryear, percentile, scope, 2)
graphing(all_data, ryear, percentile)

print("1 week")
all_data = using_all_weeks("01-01", ryear, percentile, scope, 1)
graphing(all_data, ryear, percentile)

#for i in all_data:
    #print(i)

4 weeks
['-0.0474, 2012/06/10 - 2012/07/15', '-0.0441, 2002/06/17 - 2002/07/22', '-0.0441, 1988/05/20 - 1988/06/24', '-0.0433, 2012/06/17 - 2012/07/22', '-0.0407, 2011/06/24 - 2011/07/29', '-0.0390, 1988/06/10 - 1988/07/15', '-0.0380, 2002/06/10 - 2002/07/15', '-0.0370, 2016/06/03 - 2016/07/08', '-0.0369, 1998/07/08 - 1998/08/12', '-0.0366, 2012/06/03 - 2012/07/08']
3 weeks
['-0.0385, 2002/06/24 - 2002/07/22', '-0.0379, 2011/06/24 - 2011/07/22', '-0.0370, 2012/06/17 - 2012/07/15', '-0.0359, 2012/06/10 - 2012/07/08', '-0.0348, 2002/06/17 - 2002/07/15', '-0.0330, 1998/07/08 - 1998/08/05', '-0.0325, 2012/06/24 - 2012/07/22', '-0.0318, 1988/05/20 - 1988/06/17', '-0.0316, 1983/06/24 - 1983/07/22', '-0.0313, 1988/05/27 - 1988/06/24']
2 weeks
['-0.0292, 1984/05/27 - 1984/06/17', '-0.0291, 2002/06/24 - 2002/07/15', '-0.0270, 2011/07/01 - 2011/07/22', '-0.0262, 2012/06/24 - 2012/07/15', '-0.0260, 1991/06/10 - 1991/07/01', '-0.0259, 2011/06/24 - 2011/07/15', '-0.0255, 2012/06/17 - 2012/07/08', '

'\nonly april to september\nevaporation, rainfall, streamflow\naverage for box instead of just single coordinate-- DONE\n90-274 (april to end of sept) -- DONE\n\ndates of top 10 dates (drops) for all 3 and see for matches for all 4 weeks\nhighlight ones that match\nstart documentation\nsummarize functions\nadd dictionary of variables\n\n\nHOW TO ORGANIZE:\nList top 10 streamflow events (list 10 separate dates, list all anyway)\nList top 10 rainfall events\nList top 10 evaporation events\n\n'